# Perceptron

A **perceptron** is a classification model that consists of a set of weights and scores for every feature, and a threshold. The perceptron multiplies each weight by its corresponding score, and adds them. If this score is greater than or equal to the threshold, then the perceptron returns **True**, or 1. If it is smaller, then it returns **False**, or the value 0.

The perceptron is a linear classifier and is limited to solving linearly separable problems.

#### Pseudocode

#### Sonar dataset

The file contains various patterns obtained by bouncing sonar signals off a metal cylinder or from rocks at various angles and under various conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock.

Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time. The integration aperture for higher frequencies occur later in time, since these frequencies are transmitted later during the chirp.

The label associated with each record contains the letter “Rock” if the object is a rock and “Mine” if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.

## Perceptron from scratch

We import simple packages for randomizing a seed, and the CSV reader class for reading the dataset

In [1]:
# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader

#### Data Preprocessing

We define our loading function, which takes a string representing the name of the CSV file as an argument. We then initialize an empty list inside the function to store the data from the CSV file. We open the file in read mode ('r'), create a csv_reader object and pass our file, then iterate through each row in the CSV file. If the row is empty, continue to the next row.

Afterwards we define a function to load the dataset using our previous function, skip the first row (header), and return all the remaining rows in the dataset.

In [2]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Exclude the first row
def load_csv_exclude_first_row(filename):
    dataset = load_csv(filename)
    return dataset[1:]  # Exclude the first row

Here we take a specified column for the current row, then strip any extra whitespace using strip(), and convert the string to float to make sure we are working with numerical values.

In [3]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

We iterate through the dataset and create a list which contains all the values from the target column. We then create a set called ***unique*** to store the unique values from the *class_values* list, to ensure that each unique value is considered only once. We initialize an empty dictionary called lookup. This dictionary will be used to map each unique string value to a unique integer. The key will be the string value, and the value will be the corresponding integer.

Finally, return the lookup dictionary.

In [4]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values) # Store unique values
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

#### Cross Validation

K-fold cross-validation is a technique for evaluating predictive models. The dataset is divided into k subsets or folds. The model is trained and evaluated k times, using a different fold as the validation set each time. Performance metrics from each fold are averaged to estimate the model's generalization performance.

For each fold, we iterate n times. Within each iteration, we create a new fold and populate it with fold_size samples randomly selected from the dataset copy.

We use randrange to select a random index from dataset_copy, and add it to the fold. We then remove it from dataset_copy to avoid duplication.

Finally, the fold is added to dataset_split.

In [5]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds) # Calculates the size of each fold
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy)) # select a random index from dataset_copy
            fold.append(dataset_copy.pop(index)) # remove from dataset_copy to avoid duplication.
        dataset_split.append(fold)
    return dataset_split

Comparing the predicted labels with the actual labels

Initialize a variable to keep track of the number of correct predictions. For each data point, check if the actual label in the actual list (at index i) matches the predicted label in the predicted list (at index i). If they match, it means the prediction is correct, and ***correct*** is incremented by 1.

After looping through all data points, calculate the accuracy percentage.

In [6]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0 # keep track of correct predictions
    for i in range(len(actual)):
        if actual[i] == predicted[i]: #  check if the actual label matches the predicted label
            correct += 1
    return correct / float(len(actual)) * 100.0 # get percentage

Call cross_validation_split with the dataset and n_folds arguments. For each fold, a training set and a test set are created. The training set is constructed by combining all the folds except the current fold. The line train_set = sum(train_set, []) is used to concatenate the lists representing each fold into a single training set list.

A separate test set is created for each fold by copying the data points from the fold. Each row in the fold is copied to the test set, and the class label (the last element in each row) is set to None to simulate a scenario where the true labels are hidden during testing.

The algorithm function is then called with the training set and test set for the current fold, we return predictions for the test set. The actual class labels for the test set are extracted from the current fold, and the accuracy_metric function is used to calculate the accuracy score by comparing the actual and predicted labels.

The accuracy score is appended to the scores list for the current fold. After looping through all the folds, we return the list of evaluation scores, one for each fold.

In [7]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds) # create a training set by combining all folds except the current fold.
        train_set.remove(fold)
        train_set = sum(train_set, []) # separate the current fold as the test set.
        test_set = list()
        for row in fold: # preparing the test set by creating a copy of each row in the fold
            row_copy = list(row)
            test_set.append(row_copy) # Add copied row to the test_set
            row_copy[-1] = None # remove the label from the row
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold] # Extract the actual labels from the current fold.
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

Here we define a function to make predictions with a set of weights. **weights[0]** is the bias term, while **weights[i + 1]** values correspond to the weights associated with the input features in **row**.

We initialize **activation** with the value of **weights[0]**, then iterate over each feature in **row**. In each iteration **activation** is updated by adding the product of **weights[i + 1]** and **row[i]**. This is the weighted sum of the features, where each feature is multiplied by its corresponding weight.

After exiting the loop, we check whether the final **activation** value is greater than or equal to **0.0**. If it is, return **1.0**, else, return **0.0**.

In [8]:
# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row) - 1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

#### Perceptron Loss Function

**Gradient descent** is an optimization algorithm used to adjust the weights and biases of a machine learning model, such as a perceptron, in order to find the best set of parameters that minimize the loss. Gradient descent iteratively updates the parameters in a way that reduces the loss until it converges to a minimum.

The loss function for a perceptron is based on the sum of errors for wrongly classified points, it's a piecewise linear function. 

> A piecewise linear function, is a mathematical function defined on a continuous range of real numbers, composed of straight-line segments, and each segment represents a linear function that includes both linear transformations and translations.
>
> In other words, a piecewise linear function is a way to represent complex functions by breaking them down into simpler linear segments, each defined over an interval.

Gradient descent will update the perceptron's weights and bias in such a way that it tries to push the loss function towards its minimum, trying to correctly classify as many data points as possible.

---
The following function is for finding the optimal set of weights for the perceptron based on a training dataset.

**l_rate** is the learning rate, which controls the step size in weight updates during the training process. It's considered a hyperparameter.

> Hyperparameters are parameters whose values control the learning process and determine the values of model parameters that a learning algorithm ends up learning.

An epoch is one complete pass through the entire training dataset. **n_epoch** is the number of training epochs. The length of the list **weights** is equal to the number of features in each data point.

Iterate for *n* epochs. For each data point in the training dataset:
- Make a prediction for the current data point **row** using the current set of weights.
- The error is calculated by subtracting the predicted value from the actual target label.
- The weight for the bias is updated by adding the product of the learning rate and the error. This gets the decision boundary closer to the correct classification.

For each feature in the data point:
- The weights for the individual features are updated similarly to account for each's contribution to the classification.

Finally, the trained weights are returned as the result of the function.


In [9]:
# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row) - 1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    return weights

Defining the perceptron function.

We store the predictions made by the Perceptron on the test data in an empty list **"predictions"**, then call the **train_weights** function to train the Perceptron on the provided training data. Return the weights learned during training.

Iterate through the test dataset. For each data point in the test dataset:
- Make a prediction for the current data point **row** using the weights learned during training.
- The prediction for the current data point is added to the **predictions** list. This list contains the Perceptron's predictions *(0 or 1)* for each data point in the test dataset.

In [10]:
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return predictions

#### Main Code

We load and prepare the data from our CSV file, exclude the first row from the dataset using the **load_csv_exclude_first_row** function, to remove the header. Then convert the feature columns from strings to floats numbers using the **str_column_to_float** function per every row on the dataset. Finally, convert class labels from strings to unique integer values.

We set **n_folds** to 3, for a 3-fold cross-validation. Set **l_rate** to 0.01 and **n_epoch** to 500 for the Perceptron.

We then perform the cross-validation and model evaluation via **evaluate_algorithm**, passing the *perceptron* function for the ***algorithm*** parameter on the **evaluate_algorithm** function. The results of the model's performance on each fold are stored in the **scores** variable.

Finally, print out the individual accuracy scores for each fold of the cross-validation, also calculate and print the mean accuracy across all folds.

In [11]:
# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = './datasets/sonar.csv'
dataset = load_csv_exclude_first_row(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)

# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [76.81159420289855, 69.56521739130434, 72.46376811594203]
Mean Accuracy: 72.947%
